# Real-Time Experimental Results Visualizer

## Setup

In [1]:
import pymem.utils.pipeline as pipeline
import plotly.graph_objects as go

In [2]:
fv_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Formula-Value Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Formula Size (MB)")
    )
)
fv_fig.add_scatter(name='Formula-Value (MB)')
fv_fig

FigureWidget({
    'data': [{'name': 'Formula-Value (MB)', 'type': 'scatter', 'uid': '4e946748-9439-4af0-8232-…

In [3]:
vo_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Value-Only Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Value Size (MB)")
    )
)
vo_fig.add_scatter(name='Value-Only (MB)')
vo_fig

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'scatter', 'uid': '1640596e-eedf-4cdc-92da-072…

In [4]:
barplt = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Size (MB)")
    )
)
barplt.add_bar(name='Value-Only (MB)')
barplt.add_bar(name='Formula-Value (MB)')
barplt

FigureWidget({
    'data': [{'name': 'Value-Only (MB)', 'type': 'bar', 'uid': '4ac626e6-62cb-4352-94cc-8129b43…

## Place your own settings here

In [5]:
# Input and output paths are relative to root directory
INTEGER_ARG = 5
OUTPUT_PATH = os.path.join("experiments", "results", "excel", "rcbs-5trials-rand-1col", "run2")
SOFFICE_DIR = "C:/Program Files/LibreOffice/program/soffice"
CONFIG_ARGS = pipeline.ConfigArgs(
    path=os.path.join("experiments", "results", "excel", "rcbs-5trials-rand-1col", "dataset")
    , inst="CompleteBipartiteSum"
    , rand=42
    , xlsx=True
    , step=10000
    , rows=0
    , cols=1
    , itrs=11
    , pool=5
)

## Run the following cell for real-time results

In [6]:
pipeline.run_vis(CONFIG_ARGS
  , OUTPUT_PATH
  , INTEGER_ARG
  , SOFFICE_DIR
  , fv_fig
  , vo_fig
  , barplt
  , 'uss'
)

 (MB)': 86.92258133333333, 'Value private (MB)': 86.92258133333333, 'Value wset (MB)': 141.709312, 'Value rss (MB)': 141.709312, 'Value uss (MB)': 62.737066666666664, 'Value vms (MB)': 86.92258133333333, 'Formula peak_nonpaged_pool (MB)': 0.06504266666666667, 'Formula peak_paged_pool (MB)': 1.5551253333333332, 'Formula peak_pagefile (MB)': 95.453184, 'Formula nonpaged_pool (MB)': 0.06380533333333334, 'Formula paged_pool (MB)': 1.514624, 'Formula peak_wset (MB)': 155.42272, 'Formula pagefile (MB)': 94.85789866666667, 'Formula private (MB)': 94.85789866666667, 'Formula wset (MB)': 154.689536, 'Formula rss (MB)': 154.689536, 'Formula uss (MB)': 69.90097066666667, 'Formula vms (MB)': 94.85789866666667}
Opening fv-90000.xlsx (trial 1)
Opening fv-90000.xlsx (trial 2)
Opening fv-90000.xlsx (trial 3)
Opening fv-90000.xlsx (trial 4)
Opening fv-90000.xlsx (trial 5)
{'Value peak_nonpaged_pool (MB)': 0.06317066666666667, 'Value peak_paged_pool (MB)': 1.533144, 'Value peak_pagefile (MB)': 89.74336,

In [7]:
# Timing notes

# EXCEL:
# large-RCBS-1col (Python)
#   Up to 1 million: Total time (HH:MM:SS): 06:56:18
# RCBS-1col (Python)
#   Up to 1 million: Total time (HH:MM:SS): 07:05:28
# RCBS-1col (VBA)
#   Up to 1 million: Total time (HH:MM:SS): 07:47:56
# RLS-1col
#   Up to 1 million: Total time (HH:MM:SS): 00:06:18
# RSCS-1col (VBA)
#   Up to 1 million: Total time (HH:MM:SS): 00:08:18
# RLS-5col
#   Up to 1 million: Total time (HH:MM:SS): 00:21:21